In [20]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,626 B in 1s (3,028 B/s)
Reading package lists... Done


In [21]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [22]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=',', header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [23]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [29]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
four_bedroom = spark.sql("SELECT ROUND(AVG(price), 2) AS AVERAGE_PRICE, YEAR(date) AS YEAR FROM home_sales WHERE bedrooms = 4 GROUP BY YEAR ORDER BY YEAR DESC")
four_bedroom.show()

+-------------+----+
|AVERAGE_PRICE|YEAR|
+-------------+----+
|    296363.88|2022|
|    301819.44|2021|
|    298353.78|2020|
|     300263.7|2019|
+-------------+----+



In [30]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
home_3bed_3bath =  spark.sql("SELECT ROUND(AVG(price), 2) AS AVERAGE_PRICE, YEAR(date_built) AS YEAR FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY YEAR(date_built) ORDER BY YEAR DESC")
home_3bed_3bath.show()

+-------------+----+
|AVERAGE_PRICE|YEAR|
+-------------+----+
|    292676.79|2017|
|    290555.07|2016|
|     288770.3|2015|
|    290852.27|2014|
|    295962.27|2013|
|    293683.19|2012|
|    291117.47|2011|
|    292859.62|2010|
+-------------+----+



In [31]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
home_3bed_3bath_2flrs_2000sqft = """
SELECT ROUND(AVG(price), 2) AS AVERAGE_PRICE, YEAR(date_built) AS YEAR_BUILT
FROM home_sales
WHERE bedrooms = 3
AND bathrooms = 3
AND floors = 2
AND sqft_living >= 2000
GROUP BY YEAR_BUILT
ORDER BY YEAR_BUILT DESC
"""
spark.sql(home_3bed_3bath_2flrs_2000sqft).show()

+-------------+----------+
|AVERAGE_PRICE|YEAR_BUILT|
+-------------+----------+
|    280317.58|      2017|
|     293965.1|      2016|
|    297609.97|      2015|
|    298264.72|      2014|
|    303676.79|      2013|
|    307539.97|      2012|
|    276553.81|      2011|
|    285010.22|      2010|
+-------------+----------+



In [32]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

home_price350000_per_view = """
SELECT ROUND(AVG(price), 2) AS AVERAGE_PRICE, view
FROM home_sales
GROUP BY view
HAVING ROUND(AVG(price), 2) >= 350000
"""
spark.sql(home_price350000_per_view).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|AVERAGE_PRICE|view|
+-------------+----+
|    788128.21|  51|
|    798684.82|  54|
|    750537.94|  69|
|    1072285.2|  87|
|    752861.18|  73|
|    767036.67|  64|
|     791453.0|  59|
|   1056336.74|  85|
|    733780.26|  52|
|     775651.1|  71|
|   1053739.33|  98|
|   1061201.42|  99|
|   1017815.92|  96|
|    1026669.5| 100|
|    695865.58|  70|
|    746877.59|  61|
|   1114042.94|  75|
|   1080649.37|  78|
|   1107839.15|  89|
|   1076205.56|  77|
+-------------+----+
only showing top 20 rows

--- 0.39288783073425293 seconds ---


In [33]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [34]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [43]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

home_price350000_per_view = """
SELECT ROUND(AVG(price), 2) AS AVERAGE_PRICE, view
FROM home_sales
GROUP BY view
HAVING ROUND(AVG(price), 2) >= 350000
"""
spark.sql(home_price350000_per_view).show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------+----+
|AVERAGE_PRICE|view|
+-------------+----+
|    788128.21|  51|
|    798684.82|  54|
|    750537.94|  69|
|    1072285.2|  87|
|    752861.18|  73|
|    767036.67|  64|
|     791453.0|  59|
|   1056336.74|  85|
|    733780.26|  52|
|     775651.1|  71|
|   1053739.33|  98|
|   1061201.42|  99|
|   1017815.92|  96|
|    1026669.5| 100|
|    695865.58|  70|
|    746877.59|  61|
|   1114042.94|  75|
|   1080649.37|  78|
|   1107839.15|  89|
|   1076205.56|  77|
+-------------+----+
only showing top 20 rows

--- 0.6184961795806885 seconds ---


In [44]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').parquet('home_sales_parquet', mode='overwrite')

In [45]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('home_sales_parquet')

In [46]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('temp_home_sales_parquet')

In [47]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

home_price350000_per_view = """
SELECT ROUND(AVG(price), 2) AS AVERAGE_PRICE, view
FROM home_sales
GROUP BY view
HAVING ROUND(AVG(price), 2) >= 350000
"""
spark.sql(home_price350000_per_view).show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|AVERAGE_PRICE|view|
+-------------+----+
|    788128.21|  51|
|    798684.82|  54|
|    750537.94|  69|
|    1072285.2|  87|
|    752861.18|  73|
|    767036.67|  64|
|     791453.0|  59|
|   1056336.74|  85|
|    733780.26|  52|
|     775651.1|  71|
|   1053739.33|  98|
|   1061201.42|  99|
|   1017815.92|  96|
|    1026669.5| 100|
|    695865.58|  70|
|    746877.59|  61|
|   1114042.94|  75|
|   1080649.37|  78|
|   1107839.15|  89|
|   1076205.56|  77|
+-------------+----+
only showing top 20 rows

--- 0.6552646160125732 seconds ---


In [48]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [49]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False